<a href="https://colab.research.google.com/github/SMunira38/RPDC_with_vgg19/blob/main/vgg19_(25_epoch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models import vgg19
import matplotlib.pyplot as plt

In [3]:
# Define paths for the dataset
data_dir = '/content/drive/MyDrive/Full Custom Data BRRI'
batch_size = 32

In [4]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# Load the entire dataset from Google Drive
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)

In [6]:
# Calculate the sizes of train and test sets
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
print("Train Size:", train_size)
print("Test Size:", test_size)

Train Size: 1120
Test Size: 480


In [7]:
# Split the dataset into train and test sets
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders for train and test sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
num_classes = len(dataset.classes)
print("Number of classes: ", num_classes)

Number of classes:  8


In [9]:
# Load the pre-trained VGG19 model and modify it for your classification task

model = vgg19(pretrained=True)
model.classifier[6] = nn.Linear(4096, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 85.8MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [10]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
# Training loop
num_epochs = 25
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions_train = 0
    total_samples_train = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_samples_train += labels.size(0)
        correct_predictions_train += (predicted == labels).sum().item()

    train_losses.append(running_loss / len(train_loader))
    train_accuracy = correct_predictions_train / total_samples_train
    train_accuracies.append(train_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {train_losses[-1]} - Training Accuracy: {train_accuracy*100:.2f}%")

print("Finished Training")

Epoch [1/25] - Loss: 1.1018008078847612 - Training Accuracy: 62.95%
Epoch [2/25] - Loss: 0.5238608224051339 - Training Accuracy: 82.32%
Epoch [3/25] - Loss: 0.26409627412046704 - Training Accuracy: 90.62%
Epoch [4/25] - Loss: 0.16683987535403244 - Training Accuracy: 93.93%
Epoch [5/25] - Loss: 0.15533917918801307 - Training Accuracy: 95.18%
Epoch [6/25] - Loss: 0.1502032042081867 - Training Accuracy: 95.45%
Epoch [7/25] - Loss: 0.03189875607910965 - Training Accuracy: 98.93%
Epoch [8/25] - Loss: 0.026567689191350448 - Training Accuracy: 98.93%
Epoch [9/25] - Loss: 0.10305454516118126 - Training Accuracy: 96.43%
Epoch [10/25] - Loss: 0.12551375904918782 - Training Accuracy: 95.89%
Epoch [11/25] - Loss: 0.05744576346395271 - Training Accuracy: 98.12%
Epoch [12/25] - Loss: 0.013770555759713586 - Training Accuracy: 99.73%
Epoch [13/25] - Loss: 0.01146274450501161 - Training Accuracy: 99.73%
Epoch [14/25] - Loss: 0.005364491200556845 - Training Accuracy: 99.91%
Epoch [15/25] - Loss: 0.00251

In [12]:
# Testing loop
test_accuracies = []

model.eval()
correct_predictions_test = 0
total_samples_test = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples_test += labels.size(0)
        correct_predictions_test += (predicted == labels).sum().item()

test_accuracy = correct_predictions_test / total_samples_test
test_accuracies.append(test_accuracy)
print(f"Testing Accuracy: {test_accuracy*100:.2f}%")

Testing Accuracy: 93.54%
